# 0. Usando Jupyter Notebook

**Modos**

* <kbd>esc</kbd> o <kbd>enter</kbd> Alternar entre modo navegación y modo edición
* <kbd>y</kbd> Cambiar contenido de celda a código
* <kbd>m</kbd> Cambiar contenido de celda a Markdown

**Navegación**

* <kbd>&uarr;</kbd> o <kbd>j</kbd> Seleccionar celda de arriba
* <kbd>&darr;</kbd> o <kbd>k</kbd> Seleccionar celda de abajo
* <kbd>f</kbd> Buscar y reemplazar

* <kbd>a</kbd> Insertar celda arriba
* <kbd>b</kbd> Insertar celda abajo
* <kbd>dd</kbd> Eliminar celda

**Ejecución**

* <kbd>ctrl</kbd> + <kbd>enter</kbd>: Ejecutar celda
* <kbd>shift</kbd> + <kbd>enter</kbd>:  Ejecutar celda y seleccionar siguiente (crea una si no existe)

**Extras**

* <kbd>p</kbd> Abrir paleta de comandos
* <kbd>h</kbd> Abrir lista de atajos

# 1. Importar dependencias

Importamos las librerías que vamos a usar. En este caso usaremos la mayor parte de los métodos definidos en la clase `CSP` por lo que importamos todo usando `from csp import *`.

Podemos importar métodos específicos como en el caso de `psource` que lo utilizaremos por su utilidad para visualizar código

Además, utilizaremos otras librerías que importaremos más adelante como `matplotlib` para visualización.

In [1]:
from csp import *
from notebook import psource
%matplotlib inline

# Ocultar advertencias de los gráficos
import warnings
warnings.filterwarnings("ignore")

# 2. Clase `CSP`

Esta clase nos permite definir un problema de satisfacción de restricciones en base a cuatro parámetros:

`CSP(variables, dominios, vecinos, restricciones)`

* variables: Recibe una lista de variables. Los tipos de cada variable pueden variar (`int` o `string`).
* dominios: Recibe un diccionario con los posibles valores (`{var1:[valor1, ...]}`)
* vecinos: Recibe un diccionario con listas de variables que participan de las restricciones (`{var1:[var2,...]}`)
* restricciones: Reicibe una función que establece una relación entre variables `A` y `B` cuando toman valores `a` y `b` respectivamente.

A continuación vemos la definición de la clase y sus métodos:

In [2]:
psource(CSP)

Como podemos ver `CSP` hereda de la clase `search.Problem`. Durante la clase dijimos que los CSPs son un caso especial de problemas de búsqueda.

## 2.1 Coloreo de mapas `MapColoringCSP`

Esta función crea una instancia de `CSP` con la restricción específica de que variables que son vecinos no pueden tener valores iguales.

También podemos ver su definición para saber qué tipo de parámetros pasar:

In [3]:
psource(MapColoringCSP)

Vemos que esta función hace uso de otra función `parse_neighbors` que nos permite crear un diccionario con TODAS las restricciones explícitas a partir de un `string` con ciertas restricciones.

```'X: Y Z; Y: Z'     =======>     {'Y': ['X', 'Z'], 'X': ['Y', 'Z'], 'Z': ['X', 'Y']}```

In [4]:
psource(parse_neighbors)

# 3. Creando nuestro CSP 

En este ejemplo definiremos un CSP parecido al analizado en clase, pero con los departamentos de Bolivia.

## 3.1 Vecinos

Podemos crear nuestro propio diccionario con una abreviación del departamento como llave (key) y una lista de los departamentos con los que limita como valores (values). (Nota: dict ~ key-value pairs).

In [5]:
vecinos_dict = {'BE': ['CO', 'LP', 'PA', 'SC'],
'CH': ['CO', 'PO', 'SC', 'TA'],
'CO': ['BE', 'CH', 'LP', 'OR', 'PO', 'SC'],
'LP': ['BE', 'CO', 'PA', 'OR'],
'OR': ['CO', 'LP', 'PO'],
'PA': ['BE', 'LP'],
'PO': ['CH', 'CO', 'OR', 'TA'],
'SC': ['BE', 'CH', 'CO'],
'TA': ['CH', 'PO']}

Podemos hacer uso de `parse_neighbors` si tenemos un `string` para crear el diccionario equivalente.

In [6]:
fronteras = """BE: CO LP PA SC;
CH: CO PO SC TA;
CO: LP OR PO SC;
LP: PA OR;
OR: PO;
PO: TA"""

vecinos = parse_neighbors(fronteras)

Estas dos formas nos van a generar el mismo resultado, como podemos ver en la comparación a continuación:

In [7]:
vecinos == vecinos_dict

True

## 3.2 Variables

Podemos asignar las llaves de nuestro diccionario a una lista que usaremos como nuestro argumento de variables:

In [8]:
departamentos = list(vecinos.keys())
display(departamentos)

['BE', 'CO', 'LP', 'PA', 'SC', 'CH', 'PO', 'TA', 'OR']

## 3.3 Dominios

Crearemos una lista con los posibles valores que podrán tomar nuestras variables. Seguiremos la misma nomenclatura que hemos usado hasta ahora:

`R = rojo, G = verde, B = azul`

In [9]:
valores = list('RGB') # Equivalente a ['R','G','B']

Podemos utilizar la lista `departamentos` previamente declarada para iterar sobre cada uno de sus elementos (es decir un bucle `forEach`, en Python sigue la siguiente sintaxis `for x in collection`) 

In [10]:
colores_dict = {departamento:valores for departamento in departamentos}
display(colores_dict)

{'BE': ['R', 'G', 'B'],
 'CO': ['R', 'G', 'B'],
 'LP': ['R', 'G', 'B'],
 'PA': ['R', 'G', 'B'],
 'SC': ['R', 'G', 'B'],
 'CH': ['R', 'G', 'B'],
 'PO': ['R', 'G', 'B'],
 'TA': ['R', 'G', 'B'],
 'OR': ['R', 'G', 'B']}

También podemos hacer uso de un tipo de dato que ha sido definido en la librería `CSP` llamado UniversalDict. Como lo dice en su definición, este diccionario nos devuelve el mismo valor para cualquier llave. Esta vez usaremos el magic `%pdoc` en vez de `psource` para ver más información.

In [11]:
%pdoc UniversalDict

In [12]:
colores = UniversalDict(valores)
display(colores)

{Any: ['R', 'G', 'B']}

Nótese que nos devolverá el mismo valor, sin importar qué llave usemos:

In [13]:
display(colores[1337], colores['batman'])

['R', 'G', 'B']

['R', 'G', 'B']

## 3.4 Restricciones

Recordemos que el argumento de restricciones recibe una función que a su vez recibe 4 argumentos, 2 variables y sus 2 valores asignados, y que contiene la lógica de la restricción. Para este problema de coloreo de mapas, usaremos la funcion `different_values_constraint`.

In [14]:
psource(different_values_constraint)

## 3.5 Uniendo las partes

Podemos crear nuestra instancia de `CSP` ya sea usando el constructor de esa misma clase o usando `MapColoringCSP`. Tenemos que asegurarnos de usar los argumentos correctos.

**Nota:** Si no definimos las variables, `CSP` las extraerá de las llaves del diccionario de dominios (revisar declaración).

In [15]:
bolivia = CSP(variables=None,domains=colores_dict,neighbors=vecinos_dict,constraints=different_values_constraint)
vars(bolivia)

{'variables': ['BE', 'CO', 'LP', 'PA', 'SC', 'CH', 'PO', 'TA', 'OR'],
 'domains': {'BE': ['R', 'G', 'B'],
  'CO': ['R', 'G', 'B'],
  'LP': ['R', 'G', 'B'],
  'PA': ['R', 'G', 'B'],
  'SC': ['R', 'G', 'B'],
  'CH': ['R', 'G', 'B'],
  'PO': ['R', 'G', 'B'],
  'TA': ['R', 'G', 'B'],
  'OR': ['R', 'G', 'B']},
 'neighbors': {'BE': ['CO', 'LP', 'PA', 'SC'],
  'CH': ['CO', 'PO', 'SC', 'TA'],
  'CO': ['BE', 'CH', 'LP', 'OR', 'PO', 'SC'],
  'LP': ['BE', 'CO', 'PA', 'OR'],
  'OR': ['CO', 'LP', 'PO'],
  'PA': ['BE', 'LP'],
  'PO': ['CH', 'CO', 'OR', 'TA'],
  'SC': ['BE', 'CH', 'CO'],
  'TA': ['CH', 'PO']},
 'constraints': <function csp.different_values_constraint(A, a, B, b)>,
 'initial': (),
 'curr_domains': None,
 'nassigns': 0}

In [16]:
bolivia_mapcoloring = MapColoringCSP(neighbors=vecinos,colors=valores)
vars(bolivia_mapcoloring)

{'variables': ['BE', 'CO', 'LP', 'PA', 'SC', 'CH', 'PO', 'TA', 'OR'],
 'domains': {Any: ['R', 'G', 'B']},
 'neighbors': defaultdict(list,
             {'BE': ['CO', 'LP', 'PA', 'SC'],
              'CO': ['BE', 'CH', 'LP', 'OR', 'PO', 'SC'],
              'LP': ['BE', 'CO', 'PA', 'OR'],
              'PA': ['BE', 'LP'],
              'SC': ['BE', 'CH', 'CO'],
              'CH': ['CO', 'PO', 'SC', 'TA'],
              'PO': ['CH', 'CO', 'OR', 'TA'],
              'TA': ['CH', 'PO'],
              'OR': ['CO', 'LP', 'PO']}),
 'constraints': <function csp.different_values_constraint(A, a, B, b)>,
 'initial': (),
 'curr_domains': None,
 'nassigns': 0}

# 3.6 Resolviendo un CSP

Nuestra principal herramienta para resolución de CSPs es `backtracking_search`, en su declaración podemos ver que acepta más parametros además de un `CSP`:
* `select_unassigned_variable`: podemos usar `mrv` para seleccionar la variable más restringida (es decir, con menos valores posibles)
* `order_domain_values`: podemos usar `lcv` para elegir el orden de asignación de valores de manera que causen menos conflictos
* `inference`: podemos usar `forward_checking` o `mac` (mantain arc consistency) para eliminar valores que causan conflictos en variables no asignadas.

A continuación, resolveremos un `CSP` usando `backtracking_search` sin ningún tipo de inferencia, ordenamiento de valores o criterio de selección de variables. Usaremos una versión modificada que imprimirá cada paso de modo que podamos observar la recursividad del algoritmo.

In [17]:
resultado_bolivia = backtracking_search_commented(bolivia_mapcoloring)

Se asignará la variable BE
Se está considerando el valor R
No se encontraron conflictos. Asignando BE = R
Llamando backtrack recursivamente
Se asignará la variable CO
Se está considerando el valor R
Hay 1 conflictos al asignar el valor R
Se está considerando el valor G
No se encontraron conflictos. Asignando CO = G
Llamando backtrack recursivamente
Se asignará la variable LP
Se está considerando el valor R
Hay 1 conflictos al asignar el valor R
Se está considerando el valor G
Hay 1 conflictos al asignar el valor G
Se está considerando el valor B
No se encontraron conflictos. Asignando LP = B
Llamando backtrack recursivamente
Se asignará la variable PA
Se está considerando el valor R
Hay 1 conflictos al asignar el valor R
Se está considerando el valor G
No se encontraron conflictos. Asignando PA = G
Llamando backtrack recursivamente
Se asignará la variable SC
Se está considerando el valor R
Hay 1 conflictos al asignar el valor R
Se está considerando el valor G
Hay 1 conflictos al asigna

# EXTRA: Visualizando nuestra solución

Crearemos una nueva clase llamada `InstruCSP` que heredará de `CSP`con la diferencia de que modificaremos los métodos `assign` y `unassign` para que agreguen las asignaciones parciales (y completa si es encontrada) a una lista `assignment_history`.

In [18]:
import copy
class InstruCSP(CSP):
    
    def __init__(self, variables, domains, neighbors, constraints):
        super().__init__(variables, domains, neighbors, constraints)
        self.assignment_history = []
        
    def assign(self, var, val, assignment):
        super().assign(var,val, assignment)
        self.assignment_history.append(copy.deepcopy(assignment))
    
    def unassign(self, var, assignment):
        super().unassign(var,assignment)
        self.assignment_history.append(copy.deepcopy(assignment))
        
def make_instru(csp):
    return InstruCSP(csp.variables, csp.domains, csp.neighbors, csp.constraints)

Inicializamos una nueva variable `CSP` con una instancia de `make_instru` que nos devolvera un `CSP` con `assignment_history`.

In [19]:
bolivia1 = make_instru(bolivia_mapcoloring)
vars(bolivia1)

{'variables': ['BE', 'CO', 'LP', 'PA', 'SC', 'CH', 'PO', 'TA', 'OR'],
 'domains': {Any: ['R', 'G', 'B']},
 'neighbors': defaultdict(list,
             {'BE': ['CO', 'LP', 'PA', 'SC'],
              'CO': ['BE', 'CH', 'LP', 'OR', 'PO', 'SC'],
              'LP': ['BE', 'CO', 'PA', 'OR'],
              'PA': ['BE', 'LP'],
              'SC': ['BE', 'CH', 'CO'],
              'CH': ['CO', 'PO', 'SC', 'TA'],
              'PO': ['CH', 'CO', 'OR', 'TA'],
              'TA': ['CH', 'PO'],
              'OR': ['CO', 'LP', 'PO']}),
 'constraints': <function csp.different_values_constraint(A, a, B, b)>,
 'initial': (),
 'curr_domains': None,
 'nassigns': 0,
 'assignment_history': []}

Ahora usaremos la versión original de `backtracking_search` para resolver este `CSP`.

In [20]:
resultado = backtracking_search(bolivia1)
display(resultado)

{'BE': 'R',
 'CO': 'G',
 'LP': 'B',
 'PA': 'G',
 'SC': 'B',
 'CH': 'R',
 'PO': 'B',
 'TA': 'G',
 'OR': 'R'}

Podemos ver accediendo la variable `nassigns` el número de variables que han sido asignadas, y la lista `assignment_history` para ver en qué orden se hicieron las asignaciones.

In [21]:
bolivia1.nassigns

9

In [22]:
bolivia1.assignment_history

[{'BE': 'R'},
 {'BE': 'R', 'CO': 'G'},
 {'BE': 'R', 'CO': 'G', 'LP': 'B'},
 {'BE': 'R', 'CO': 'G', 'LP': 'B', 'PA': 'G'},
 {'BE': 'R', 'CO': 'G', 'LP': 'B', 'PA': 'G', 'SC': 'B'},
 {'BE': 'R', 'CO': 'G', 'LP': 'B', 'PA': 'G', 'SC': 'B', 'CH': 'R'},
 {'BE': 'R', 'CO': 'G', 'LP': 'B', 'PA': 'G', 'SC': 'B', 'CH': 'R', 'PO': 'B'},
 {'BE': 'R',
  'CO': 'G',
  'LP': 'B',
  'PA': 'G',
  'SC': 'B',
  'CH': 'R',
  'PO': 'B',
  'TA': 'G'},
 {'BE': 'R',
  'CO': 'G',
  'LP': 'B',
  'PA': 'G',
  'SC': 'B',
  'CH': 'R',
  'PO': 'B',
  'TA': 'G',
  'OR': 'R'}]

Importamos la librería `matplotlib` y su módulo `pyplot` es bindeado a `plt` para hacer visualizaciones. También importamos `networkx` para crear nuestra estructura de grafo. Por último, la librería `time` será usada para agregar una pausa entre pasos.

In [23]:
%matplotlib inline
import networkx as nx
import matplotlib.pyplot as plt
import matplotlib
import time

In [24]:
def make_update_step_function(graph, instru_csp):
    
    def draw_graph(graph):
        # create networkx graph
        G=nx.Graph(graph)
        # draw graph
        pos = nx.spring_layout(G,k=0.15)
        return (G, pos)
    
    G, pos = draw_graph(graph)
    
    def update_step(iteration):
        # here iteration is the index of the assignment_history we want to visualize.
        current = instru_csp.assignment_history[iteration]
        #  We convert the particular assignment to a default dict so that the color for nodes which 
        # have not been assigned defaults to black.
        current = defaultdict(lambda: 'Black', current)

        # Now we use colors in the list and default to black otherwise.
        colors = [current[node] for node in G.node.keys()]
        # Finally drawing the nodes.
        nx.draw(G, pos, node_color=colors, node_size=500)

        labels = {label:label for label in G.node}
        # Labels shifted by offset so as to not overlap nodes.
        label_pos = {key:[value[0], value[1]+0.03] for key, value in pos.items()}
        nx.draw_networkx_labels(G, label_pos, labels, font_size=20)

        # show graph
        plt.show()

    return update_step  # <-- this is a function

def make_visualize(slider):
    ''' Takes an input a slider and returns 
        callback function for timer and animation
    '''
    
    def visualize_callback(Visualize, time_step):
        if Visualize is True:
            for i in range(slider.min, slider.max + 1):
                slider.value = i
                time.sleep(float(time_step))
    
    return visualize_callback

En esta parte, lo importante es notar que `make_update_step_function` devuelve una función. Es decir, al asignarla a otro variable, le estamos dando su referencia que será utilizada para aceptar argumentos. `update_step` asignará los colores adecuados a las variables que hayan sido asignadas y negro si no han sido asignadas.

In [25]:
step_func = make_update_step_function(vecinos, bolivia1)

In [26]:
matplotlib.rcParams['figure.figsize'] = (9.0, 9.0) # Definimos las dimensiones de nuestro visualización

In [27]:
import ipywidgets as widgets
from IPython.display import display

iteration_slider = widgets.IntSlider(min=0, max=len(bolivia1.assignment_history)-1, step=1, value=0)
w=widgets.interactive(step_func,iteration=iteration_slider)
display(w)

visualize_callback = make_visualize(iteration_slider)

visualize_button = widgets.ToggleButton(description = "Visualizar", value = False)
time_select = widgets.ToggleButtons(description='Retraso:',options=['0', '0.1', '0.2', '0.5', '0.7', '1.0'])

a = widgets.interactive(visualize_callback, Visualize = visualize_button, time_step=time_select)
display(a)

interactive(children=(IntSlider(value=0, description='iteration', max=8), Output()), _dom_classes=('widget-int…

interactive(children=(ToggleButton(value=False, description='Visualizar'), ToggleButtons(description='Retraso:…

# Tarea

Problema 6.9 pp. 232


> Generar instancias aleatorias de problemas de coloreo de mapas de la siguiente forma:
> 1. Dispersar $n$ puntos dentro en el cuadrado unitario $\{(0,0), (0,1), (1,1), (1, 0)\}$
> 2. Seleccionar un punto $X$ aleatoriamente
> 3. Conectar $X$ con una línea recta al punto más cercano $Y$ de modo que $X$ no haya estado conectado previamente a $Y$ y que no se cruce otras líneas existentes. Los puntos representarán regiones del mapa y las lineas conectan vecinos.
> 4. Buscar los $k$-coloreos de cada mapa, para $k=3$ y $k=4$ usando min-conflicts, backtracking, backtracking con forward checking, y backtracking con MAC.
> 5. Construir una tabla con los tiempos de ejecución promedio para cada algoritmo para valores de $n$ hasta el valor más alto que se pueda obtener. Comentar sus resultados.
